In [1]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

In [3]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [4]:
df_books.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271379 entries, 0 to 271378
Data columns (total 3 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   isbn    271379 non-null  object
 1   title   271379 non-null  object
 2   author  271377 non-null  object
dtypes: object(3)
memory usage: 6.2+ MB


In [5]:
df_ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1149780 entries, 0 to 1149779
Data columns (total 3 columns):
 #   Column  Non-Null Count    Dtype  
---  ------  --------------    -----  
 0   user    1149780 non-null  int32  
 1   isbn    1149780 non-null  object 
 2   rating  1149780 non-null  float32
dtypes: float32(1), int32(1), object(1)
memory usage: 17.5+ MB


In [6]:
df_books.dropna(inplace=True)

df_ratings.dropna(inplace=True)

In [7]:
df_books.drop_duplicates(inplace=True)

df_ratings.drop_duplicates(inplace=True)

In [8]:
number_of_reviews_per_user = df_ratings['user'].value_counts()

users_with_200_reviews_or_more = number_of_reviews_per_user.loc[
    number_of_reviews_per_user >= 200].index

number_of_reviews_per_book = df_ratings['isbn'].value_counts()

books_with_100_reviews_or_more = number_of_reviews_per_book.loc[
    number_of_reviews_per_book >= 100].index

In [9]:
df_ratings = df_ratings.loc[
    (df_ratings['user'].isin(users_with_200_reviews_or_more)) &
    (df_ratings['isbn'].isin(books_with_100_reviews_or_more))]

In [10]:
df_books.info()

<class 'pandas.core.frame.DataFrame'>
Index: 271377 entries, 0 to 271378
Data columns (total 3 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   isbn    271377 non-null  object
 1   title   271377 non-null  object
 2   author  271377 non-null  object
dtypes: object(3)
memory usage: 8.3+ MB


In [11]:
df_ratings.info()

<class 'pandas.core.frame.DataFrame'>
Index: 49781 entries, 1456 to 1147441
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   user    49781 non-null  int32  
 1   isbn    49781 non-null  object 
 2   rating  49781 non-null  float32
dtypes: float32(1), int32(1), object(1)
memory usage: 1.1+ MB


In [12]:
df_books_with_ratings = df_ratings.join(
    df_books.set_index('isbn'), on='isbn').dropna()

df_books_with_ratings.drop(['isbn'], axis=1, inplace=True)

df_books_with_ratings.reset_index(drop=True, inplace=True)

df_books_with_ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49517 entries, 0 to 49516
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   user    49517 non-null  int32  
 1   rating  49517 non-null  float32
 2   title   49517 non-null  object 
 3   author  49517 non-null  object 
dtypes: float32(1), int32(1), object(2)
memory usage: 1.1+ MB


In [13]:
df_books_with_ratings = df_books_with_ratings.pivot_table(
    index=['title', 'author'], columns='user', values='rating', sort=False
    ).fillna(0)

df_books_with_ratings.reset_index(inplace=True)

df_books_with_ratings.rename_axis(None, axis=1, inplace=True)

df_books_with_ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 681 entries, 0 to 680
Columns: 890 entries, title to 275970
dtypes: float32(888), object(2)
memory usage: 2.3+ MB


In [14]:
authors = df_books_with_ratings['author'].to_numpy().reshape(-1, 1)
user_ratings = df_books_with_ratings.iloc[:, 2:]

encoder = OneHotEncoder().fit(authors)

book_matrix = np.concatenate(
    (encoder.transform(authors).toarray(), user_ratings), axis=1)

In [15]:
# function to return recommended books - this will be tested
def get_recommends(book = ""):
  recommended_books = [book, []]

  book = df_books_with_ratings.loc[df_books_with_ratings['title'] == book]
  book = np.append(encoder.transform(
      np.array(book['author']).reshape(-1, 1)).toarray(), book.iloc[:, 2:])

  knn = NearestNeighbors(n_neighbors=6, metric='cosine').fit(book_matrix)

  nn = knn.kneighbors(book.reshape(1, -1))

  for i in range(1, 6):
    recommended_book = df_books_with_ratings.iloc[nn[1][0][i]]['title']

    recommended_books[1].append([recommended_book, nn[0][0][i]])

  recommended_books[1].reverse()

  return recommended_books

In [16]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2):
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", [["I'll Be Seeing You", 0.801746825909877], ['The Weight of Water', 0.7710496334084845], ['The Surgeon', 0.7701682263650189], ['I Know This Much Is True', 0.767815567839601], ['The Lovely Bones: A Novel', 0.7235408977208326]]]
You passed the challenge! 🎉🎉🎉🎉🎉
